In [1]:
import optuna
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)
from sklearn.feature_selection import RFE
from imblearn.combine import SMOTETomek, SMOTEENN

#URL
df1 = pd.read_csv('url_features.csv').sort_values(by='url')
# df1 = pd.get_dummies(df1, columns=['tld_type'], prefix='tld')
# X1 = df1.drop(['url','label'], axis=1)
# y1 = df1['label']

#HTML
df2 = pd.read_csv('new_html_features.csv').sort_values(by='url')
# X2 = df2.drop(['label', 'url'], axis=1)
# y2 = df2['label']

df3 = pd.merge(df1, df2, on="url")


#Check to make sure the two labels are identical
# (df3['label_x'] == df3['label_y']).all()



df3 = df3.rename(columns={'label_x': 'label'})

df3 = pd.get_dummies(df3, columns=['tld_type'], prefix='tld')
# X3 = df3.drop(['url','label', 'label_y'], axis=1)
# y3 = df3['label']


# 25% - 174,268
n_legit = 168040
n_phish = 5228

legit_df3 = df3[df3['label'] == 0]
phish_df3 = df3[df3['label'] == 1]

sampled_legit3 = legit_df3.sample(n=n_legit, random_state=42)
sampled_phish3 = phish_df3.sample(n=n_phish, random_state=42)


reduced_df3 = pd.concat([sampled_legit3, sampled_phish3]).sample(frac=1, random_state=42).reset_index(drop=True)

X3 = reduced_df3.drop(['url','label', 'label_y'], axis=1)
y3 = reduced_df3['label']

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, stratify=y3, test_size=0.2, random_state=42)

models = {
    'RandomForest': RandomForestClassifier,
    'DecisionTree': DecisionTreeClassifier,
    'LightGBM': LGBMClassifier,
    'XGBoost': XGBClassifier,
    'KNN': KNeighborsClassifier
}


def objective(trial, model_name, X_train, y_train, X_test, y_test):
    if model_name == 'RandomForest':
        params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 500),
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.8, 0.5]),
        'n_jobs': -1
    }
    elif model_name == 'DecisionTree':
        params = {
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy'])
        
    }
    elif model_name == 'LightGBM':
        params = {
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
        'n_jobs': -1
    }
    elif model_name == 'XGBoost':
        params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'n_jobs': -1
    }
    elif model_name == 'KNN':
        params = {
        'n_neighbors': trial.suggest_int('n_neighbors', 1, 30),
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
        'p': trial.suggest_int('p', 1, 2),
        'n_jobs': -1
    }

    model = models[model_name](**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return f1_score(y_test, y_pred)



X_trains = [X_train3]
y_trains = [y_train3]
X_tests = [X_test3]
y_tests = [y_test3]




best_params = {}



for i, (X_train, y_train, X_test, y_test) in enumerate(zip(X_trains, y_trains, X_tests, y_tests)):
    dataset_results = {}
    print(f'Optimizing dataset {i+1}')

    for model_name in models.keys():
        print(f'Tuning {model_name}')
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, model_name, X_train, y_train, X_test, y_test), n_trials=200)

        dataset_results[model_name] = {
            'best_params': study.best_params,
            'best_score': study.best_value
        }

    best_params[f'dataset_{i+1}'] = dataset_results


print(best_params)


C:\Users\Ricky\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-22 20:54:19,143] A new study created in memory with name: no-name-ca3dec44-1846-4b25-9a92-fb7e13188c95


Optimizing dataset 1
Tuning RandomForest


[I 2025-06-22 20:58:19,265] Trial 0 finished with value: 0.709531866892273 and parameters: {'n_estimators': 306, 'max_depth': 54, 'min_samples_split': 3, 'min_samples_leaf': 8, 'max_features': 0.5}. Best is trial 0 with value: 0.709531866892273.
[I 2025-06-22 20:58:31,301] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 441, 'max_depth': 33, 'min_samples_split': 20, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 0 with value: 0.709531866892273.
[I 2025-06-22 20:58:40,430] Trial 2 finished with value: 0.0 and parameters: {'n_estimators': 403, 'max_depth': 26, 'min_samples_split': 12, 'min_samples_leaf': 15, 'max_features': 'log2'}. Best is trial 0 with value: 0.709531866892273.
[I 2025-06-22 20:59:04,543] Trial 3 finished with value: 0.4863349684653118 and parameters: {'n_estimators': 296, 'max_depth': 53, 'min_samples_split': 20, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.709531866892273.
[I 2025-06-22 21:05:27,029] Tr

Tuning DecisionTree


[I 2025-06-23 00:42:53,341] Trial 0 finished with value: 0.23293172690763053 and parameters: {'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 11, 'criterion': 'entropy'}. Best is trial 0 with value: 0.23293172690763053.
[I 2025-06-23 00:42:59,671] Trial 1 finished with value: 0.6586036283672347 and parameters: {'max_depth': 18, 'min_samples_split': 10, 'min_samples_leaf': 7, 'criterion': 'gini'}. Best is trial 1 with value: 0.6586036283672347.
[I 2025-06-23 00:43:01,507] Trial 2 finished with value: 0.022577610536218252 and parameters: {'max_depth': 4, 'min_samples_split': 5, 'min_samples_leaf': 17, 'criterion': 'entropy'}. Best is trial 1 with value: 0.6586036283672347.
[I 2025-06-23 00:43:06,112] Trial 3 finished with value: 0.648471615720524 and parameters: {'max_depth': 18, 'min_samples_split': 16, 'min_samples_leaf': 16, 'criterion': 'entropy'}. Best is trial 1 with value: 0.6586036283672347.
[I 2025-06-23 00:43:09,647] Trial 4 finished with value: 0.5392450569203115 

...

Tuning XGBoost


[I 2025-06-23 01:09:07,040] Trial 0 finished with value: 0.7088060507833603 and parameters: {'n_estimators': 202, 'max_depth': 12, 'learning_rate': 0.1120783986855388, 'subsample': 0.764761246847474, 'colsample_bytree': 0.9630521880554643, 'gamma': 2.9823479661131618}. Best is trial 0 with value: 0.7088060507833603.
[I 2025-06-23 01:09:11,052] Trial 1 finished with value: 0.6840317100792752 and parameters: {'n_estimators': 105, 'max_depth': 8, 'learning_rate': 0.05351791936232342, 'subsample': 0.9958305295451051, 'colsample_bytree': 0.70762953476963, 'gamma': 0.6164743396194095}. Best is trial 0 with value: 0.7088060507833603.
[I 2025-06-23 01:09:15,496] Trial 2 finished with value: 0.6895027624309392 and parameters: {'n_estimators': 162, 'max_depth': 12, 'learning_rate': 0.08319272195658049, 'subsample': 0.8596029550183856, 'colsample_bytree': 0.6910678414597071, 'gamma': 3.7661046202026616}. Best is trial 0 with value: 0.7088060507833603.
[I 2025-06-23 01:09:20,970] Trial 3 finished 

Tuning KNN


[I 2025-06-23 01:43:55,649] Trial 0 finished with value: 0.7635239567233385 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 0.7635239567233385.
[I 2025-06-23 01:44:18,918] Trial 1 finished with value: 0.6692181519956261 and parameters: {'n_neighbors': 22, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 0.7635239567233385.
[I 2025-06-23 01:50:15,396] Trial 2 finished with value: 0.7213800313643491 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 0.7635239567233385.
[I 2025-06-23 01:50:37,976] Trial 3 finished with value: 0.762493560020608 and parameters: {'n_neighbors': 10, 'weights': 'distance', 'p': 2}. Best is trial 0 with value: 0.7635239567233385.
[I 2025-06-23 01:56:33,914] Trial 4 finished with value: 0.6591154261057174 and parameters: {'n_neighbors': 29, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 0.7635239567233385.
[I 2025-06-23 02:02:29,539] Trial 5 finished w

{'dataset_1': {'RandomForest': {'best_params': {'n_estimators': 206, 'max_depth': 95, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt'}, 'best_score': 0.778131634819533}, 'DecisionTree': {'best_params': {'max_depth': 20, 'min_samples_split': 11, 'min_samples_leaf': 2, 'criterion': 'entropy'}, 'best_score': 0.7091186958736627}, 'LightGBM': {'best_params': {'num_leaves': 96, 'max_depth': 15, 'learning_rate': 0.16099354321038145, 'n_estimators': 292, 'subsample': 0.995245362847253, 'colsample_bytree': 0.883261087073046, 'min_child_samples': 25}, 'best_score': 0.7744282744282744}, 'XGBoost': {'best_params': {'n_estimators': 281, 'max_depth': 14, 'learning_rate': 0.0888992418241481, 'subsample': 0.9134952932290352, 'colsample_bytree': 0.6533354924947662, 'gamma': 0.25380562179671745}, 'best_score': 0.7748132337246532}, 'KNN': {'best_params': {'n_neighbors': 18, 'weights': 'distance', 'p': 1}, 'best_score': 0.7694687983496648}}}
